# What does thid notebook do?  
using agglomerative clustering to find posible decision points in all notebooks  



# Agglomerative Clustering

In [ ]:
%matplotlib notebook

In [ ]:
import os
from tqdm import tqdm
import numpy as np
from sklearn import cluster
import scipy.cluster.hierarchy as sch
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

import pdb
import json
import sklearn
import types

from importlib import import_module

from extract_func import process_file

from sklearn import *

import inspect

import types

In [ ]:
EXPANSION = True

In [ ]:
class Tree(object):
    """tree sturcture to record currently found decision points"""
    def __init__(self, lib, funcs = []):
        """init a tree with a list of functions or []"""
        super(Tree, self).__init__()
        self.lib = lib
        self.root = {}
        for f in funcs:
            self.add_func(f)
    
    def add_func(self, func):
        """add function to tree"""
        tokens = func.split('.')
        cur_root = self.root
        for i, t in enumerate(tokens):
            if t not in cur_root:
                cur_root[t] = {}
            cur_root = cur_root[t]
            
    def add_funcs(self, funcs):
        """add functions to tree"""
        for f in funcs:
            self.add_func(f)

    def check_node(self, node):
        """delete leaves if very deep but not wide"""
        if len(node)<3:
            return {}
        else:
            node = {k: self.check_node(node[k]) for k in node}
            return node
    
    def cut_tree(self):
        self.cut_root = {self.lib: self.check_node(self.root[self.lib])}
        
    def tolist(self, prefix, tree):
        results = []
        for k in tree:
            if tree[k] =={}:
                results.append('{}.{}'.format(prefix,k) if prefix!='' else k)
            else:
                results+=self.tolist('{}.{}'.format(prefix,k) if prefix!='' else k, tree[k])
        return results
        
        

In [ ]:
def is_parent_child(parent_func, child_func):
    if len(parent_func)>=len(child_func):
        return False
    if child_func.startswith(parent_func):
        if len(child_func.split('.'))>len(parent_func.split('.')):
            return True
        else:
            return False
    else:
        return False

In [ ]:
def search_func(mod, func):
    tokens = func.split('.')
    prefix = mod.__name__
    for t in tokens[1:]:
        sub = getattr(mod, t)
        prefix +='.{}'.format(t)
        if type(sub)==types.ModuleType or type(sub)==types.ClassType:
            mod = sub
        elif type(sub)==types.FunctionType:
            break
        else:
            assert ValueError, type(sub)
    return prefix

In [ ]:
def cut_tree(obj, trees, threshold, expansion_threshold = 0.5):
    new_trees = {}
    
    modules = [item[0] for item in inspect.getmembers(obj, inspect.ismodule)]
    classes = [item[0] for item in inspect.getmembers(obj, inspect.isclass)]
    functions = [item[0] for item in inspect.getmembers(obj, inspect.isfunction)]
    
    members = trees.keys()
    
    actual_modules = [m for m in members if m in modules]
    actual_classes = [m for m in members if m in classes]
    actual_functions = [m for m in members if m in functions]
    
    module_proportion = len(actual_modules)/(len(modules)+0.0001)
    class_proportion = len(actual_classes)/(len(classes)+0.0001)
    function_proportion = len(actual_functions)/(len(functions)+0.0001)
    
    if module_proportion>threshold:
        for t in actual_modules:
            edited_tree = cut_tree(getattr(obj, t), trees[t], threshold, expansion_threshold)
            if len(edited_tree)==0:
                continue
            new_trees[t] = edited_tree
    if class_proportion>threshold:
        for t in actual_classes:
            new_trees[t] = cut_tree(getattr(obj, t), trees[t], threshold, expansion_threshold)
    if function_proportion>threshold:
        for t in actual_functions:
            new_trees[t] = cut_tree(getattr(obj, t), trees[t], threshold, expansion_threshold)
    if EXPANSION and class_proportion>expansion_threshold:
        for c in classes:
            if c not in actual_classes and not c.startswith('_'):
                new_trees[c] = {}
    if EXPANSION and function_proportion>expansion_threshold:
        for f in functions:
            if f not in actual_functions and not f.startswith('_'):
                new_trees[f] = {}
    return new_trees

In [ ]:
not_root = []
sets = []
has_root = []

def find_root_in_cluster(cluster):
    """
    find a representative function in a cluster like:
     ['sklearn.preprocessing.MinMaxScaler',
      'sklearn.preprocessing.MinMaxScaler.fit_transform',
      'sklearn.preprocessing.MinMaxScaler.transform']
    return a list []
    """
    cluster = sorted(cluster)
    last_root = '*'
    cluster_roots = []
    for f in cluster:
        if not f.startswith(last_root):
            cluster_roots.append(f)
            last_root = f
        elif f not in has_root:
            has_root.append(f)
    cluster_roots = [r for r in cluster_roots if not any([is_parent_child(root, r) for root in roots])]
    return cluster_roots
            

    

暂时没有用，但是请先保留

# delete loops:
eg  
**sklearn.tree.DecisionTreeClassifier.fit.fit.fit**  
is equal to   
**sklearn.tree.DecisionTreeClassifier.fit**  
  
the only difference is that the first one is called multiple times  

In [ ]:
def delete_loop(func):
    """
    delete continue loops in a function
    simplify **sklearn.tree.DecisionTreeClassifier.fit.fit.fit** 
          to **sklearn.tree.DecisionTreeClassifier.fit**
    """
    tokens = func.split('.')
    new_tokens = []
    for t in tokens:
        if new_tokens == [] or t!=new_tokens[-1]:
            new_tokens.append(t)
    return '.'.join(new_tokens)

In [ ]:
with open('./file_funcs.json','r') as f:
    data = json.load(f)

In [ ]:
# nb_path = '/projects/bdata/jupyter/target'
nb_path = '/home/gezhang/data/jupyter/target'
notebooks = os.listdir(nb_path)

In [ ]:
sklearn_notebooks = []
for i, nb in enumerate(notebooks):
    if i%10000==0:
        print(i)
    with open(os.path.join(nb_path, nb),'r') as f:
#         print('--')
        content = f.read()
    tokens = content.split()
    if 'sklearn' in tokens:
        sklearn_notebooks.append(nb)
    

In [ ]:
func2vector = {}
vector_size = len(sklearn_notebooks)
err_files = []
sklearn_counter = {}
for i, nb in enumerate(sklearn_notebooks):
    if i%10000 == 0:
        print('Log: {} notebooks processed'.format(i))
    funcs = []
    if nb in data:
        funcs = data[nb]["funcs"]
        linenos = data[nb]["linenos"]
    funcs = [func for func in funcs if func.startswith('sklearn.')]
    
    funcs = set(funcs)
    for func in funcs:
        if func not in func2vector:
            func2vector[func] = np.zeros(vector_size)
        if func not in sklearn_counter:
            sklearn_counter[func]=0
        func2vector[func][i] = 1
        sklearn_counter[func]+=1

In [ ]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [ ]:
a = 1- cooccur_matrix/cooccur_matrix.max()

In [ ]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.996, affinity="precomputed", linkage="average").fit(a)

In [ ]:
model.n_clusters_

In [ ]:
clusters = {}
for i, l in enumerate(model.labels_):
    if l not in clusters:
        clusters[l] = []
    clusters[l].append(idx2func[i])
clusters = list(clusters.values())

In [ ]:
clusters

In [ ]:
roots = []
for c in tqdm(clusters):
    funcs = find_root_in_cluster(c)
    roots+=funcs
    

In [ ]:
sorted(roots)

In [ ]:
len(roots)

In [ ]:
# 删去representatives 中还有sub 关系的函数（只相差一级的话）
dp = []
for f in sorted(roots):
    if dp==[] or len(dp[-1].split('.'))<3:
        dp.append(f)
    elif not is_parent_child(dp[-1], f):
        dp.append(f)
    else:
        continue

In [ ]:
len(dp)

In [ ]:
dp

删除不合法的函数

In [ ]:
temp = []
for f in dp:
    tokens = f.split('.')
    try:
        getattr(sklearn, tokens[1])
        temp.append(f)
    except Exception as e:
        print(e)
temp

删除上面这个cell，保留下面这个，看是否可以

In [ ]:
new_temp = []
for f in dp:
    tokens = f.split('.')
    try:
        getattr(sklearn, tokens[1])
        mod = import_module('sklearn.{}'.format(tokens[1]))
        getattr(mod, tokens[2])
        new_temp.append(f)
    except Exception as e:
        print(e)
new_temp

In [ ]:
search_func(sklearn, "sklearn.base.BaseEstimator.get_params")

上面两个temp new_temp 的筛选是不是在search_func 中都可以实现？

In [ ]:
# delete all incorrect functions
final_dp = []
for f in new_temp:
    try:
        print('-'*20)
        corrected_func = search_func(sklearn, f)
        if f!=corrected_func:
            print(f)
            print(corrected_func)
        final_dp.append(corrected_func)
    except Exception as e:
        print(f)
        print(e)
        

In [ ]:
final_dp

In [ ]:
# delete all functions start with "_"
all_dp = []
for f in final_dp:
    tokens = f.split('.')
    if any([t.startswith('_') for t in tokens]):
        continue
    else:
        all_dp.append(f)

In [ ]:
len(all_dp)

In [ ]:
all_dp

把function list 转为树，进行剪枝和扩充

In [ ]:

tree = Tree(sklearn, all_dp)


In [ ]:
dp1 = sorted(tree.tolist('',{"sklearn":cut_tree(sklearn, tree.root['sklearn'],0.05,0.2)}))

In [ ]:
len(dp1)

In [ ]:
for f in dp1:
    if f not in dp2:
        print(f)

In [ ]:
for f in all_dp:
    if f not in dp1:
        print(f)

In [ ]:
sorted(dp1)

In [ ]:
for f in sorted(dp1):
    print(f)